In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/AIFFEL_quest_rs/Exploration/Quest01/

/content/drive/MyDrive/AIFFEL_quest_rs/Exploration/Quest01


In [3]:
!git pull origin main

From https://github.com/nagujean/AIFFEL_quest_rs
 * branch            main       -> FETCH_HEAD
Already up to date.


In [ ]:
import os
print(os.getcwd())

/content/drive/MyDrive/AIFFEL_quest_rs/Exploration/Quest01


In [12]:
!ls

 datasets	  pytoch_2.ipynb	   pytorch_1.ipynb
 models		  pytoch_3.ipynb	   Quest01.ipynb
 non_linear.csv  'pytoch_3.ipynb의 사본'   README.md


In [4]:
#필요 라이브러리

import torch #pytorch
import pandas as pd #csv 파일 로드
from torch import nn #라이버리 내의 neural networks 모듈,Loss/activation function
from torch import optim # optimizer
from torch.utils.data import Dataset, DataLoader, random_split #데이터유틸리티

In [14]:
# 사용자 정의 데이터 셋

 # 새로운 클래스 초기화
class CustomDataset(Dataset): #새로운 클래스 정의 -> Dataset 상속받음
    def __init__(self, file_path): #객체 초기화 메서드
        df = pd.read_csv(file_path) #파일일고 데이터 프레임 저장
        self.x1 = df.iloc[:, 0].values #데이터의 첫열 저장
        self.x2 = df.iloc[:, 1].values
        self.x3 = df.iloc[:, 2].values
        self.y = df.iloc[:, 3].values #데이터의 4번열 타겟 저장
        self.length = len(df) #테이터 총 행의 수 , 전체크기


 # X값과 y값 반환 데이터 구성(혹은 변환)
    def __getitem__(self, index): #
        x = torch.FloatTensor([self.x1[index], self.x2[index], self.x3[index]])
        y = torch.FloatTensor([int(self.y[index])])
        return x, y

#데이터 전체 크기 반환, 이걸알야야 배치등을 계산할 수 있음
    def __len__(self):
        return self.length

In [15]:
# 사용자 정의 모델

class CustomModel(nn.Module): #신경망 모듈을 상속 받아 CustomModel 정의
    def __init__(self): # 모델 객체 초기화
        super().__init__() # 부모클래스 초기화 메서드 호출
        self.layer = nn.Sequential(  #여러 레이어를 순차 연결하는 컨테이너 정의
          nn.Linear(3, 1), # 입력3개 출력 1개인 '선형 변환' 레이어(가중치 학습)
          nn.Sigmoid() #활성화 함수 #0,1 사이 값으로 출력하는 '시그모이드 활성화' 함수
        )

    def forward(self, x): #모델의 순전파 정의 메서드
        x = self.layer(x) #정의된 레이러를 입력x에 순서대로 적용
        return x #최종 반환값

In [16]:
# 데이터 셋과 데이터 로더

 # 데이터셋 로드 및 크기 정의
dataset = CustomDataset("./datasets/binary.csv") #csv파일로 부터 데이터 객체 생성
dataset_size = len(dataset) #전체 데이터 수 개수 확인
train_size = int(dataset_size * 0.8) #훈련용 데이터 사이즈 지정
validation_size = int(dataset_size * 0.1) #검증용 데이터 사이즈 지정
test_size = dataset_size - train_size - validation_size #테스트 데이터 사이즈 지정

 # 데이터 셋 분할
train_dataset, validation_dataset, test_dataset = random_split(dataset, [train_size, validation_size, test_size], torch.manual_seed(4))
 # 데이터로더 생성
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=4, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=True, drop_last=True)

In [17]:
# GPU 연산 적용

device = "cuda" if torch.cuda.is_available() else "cpu" #디바이스 gpu/cpu 설정
model = CustomModel().to(device) #모델 인스턴스 생성 및 디바이스 이동
criterion = nn.BCELoss().to(device) #손실함수 이진교차 엔트로피 정의 및 디바이스 이동
optimizer = optim.SGD(model.parameters(), lr=0.0001) # 옵티마이저 SGD 정의

In [18]:
# 학습 진행

#체크 포인트 지정
checkpoint = 1
 #에포크 지정
for epoch in range(10000): #진행할 에포크 수
    cost = 0.0 # 누적손실 초기화

 #데이터로더를 통해 미니 배치 단위로 데이터 로드 및 학습
    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x) #모델에 입력을 넣어 순전파 실행
        loss = criterion(output, y) #손실 계산

        optimizer.zero_grad() #이전 역전파에서 계산된 가중치를 0초기화
        loss.backward() #역전파:손실에 대한 모델 파라미터들의 가중치 계산
        optimizer.step() #계산된 가중치를 모델 파라미터 업데이트

        cost += loss #현재 미니배치의 손실을 에포크의 총 손실에 누적

    cost = cost / len(train_dataloader) # 한 에포크의 평균 손실 계산

#체크포인트
    if (epoch + 1) % 1000 == 0:
        torch.save(
            {
                "model": "CustomModel",
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "cost": cost,
                "description": f"CustomModel 체크포인트-{checkpoint}",
            },
            f"./models/checkpoint-{checkpoint}.pt",
        )
        checkpoint += 1


 # 학습 진행 상황 출력
    if (epoch + 1) % 1000 == 0: # 1천 단위 출력
        print(f"Epoch : {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost:.3f}")

Epoch : 1000, Model : [Parameter containing:
tensor([[ 0.0028, -0.0006,  0.0036]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.0949], device='cuda:0', requires_grad=True)], Cost : 0.680
Epoch : 2000, Model : [Parameter containing:
tensor([[0.0034, 0.0005, 0.0041]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([-0.0346], device='cuda:0', requires_grad=True)], Cost : 0.666
Epoch : 3000, Model : [Parameter containing:
tensor([[0.0045, 0.0017, 0.0051]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([-0.1609], device='cuda:0', requires_grad=True)], Cost : 0.652
Epoch : 4000, Model : [Parameter containing:
tensor([[0.0052, 0.0017, 0.0053]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([-0.2843], device='cuda:0', requires_grad=True)], Cost : 0.636
Epoch : 5000, Model : [Parameter containing:
tensor([[0.0059, 0.0036, 0.0064]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([-0.4046], 

In [ ]:
# 모델 평가

 #모델 평가 모드 설정
with torch.no_grad(): # 기울기 계산 비활성화 자동미분 기능 사용하지 않도록 메모리사용량 줄여 추론에 적합한 상태로
    model.eval() # 모델 평가모드로 전환
    for x, y in validation_dataloader: # 검증 데이터 로더 미니 배치 가져옴
        x = x.to(device) #입력데이터를 설정된 디바이스로 이동
        y = y.to(device) #레이블 데이터를 설정된 디바이스로 이동

        outputs = model(x) #모델에 입력데이터를 넣어서 순전파 실행
 #예측 결과 출력 및 이진 분류 임계값 적용
        print(outputs) #모델의 원본 예측값 시그모이드 통과후 0~1사이 확률값 출력
        #예측값이 0.5 이상이면 True, 아니면 False로 변환하여 이진 분류 결과 출력
        print(outputs >= torch.FloatTensor([0.5]).to(device))
        print("--------------------")

tensor([[0.6707],
        [0.7298],
        [0.7368],
        [0.7800]], device='cuda:0')
tensor([[True],
        [True],
        [True],
        [True]], device='cuda:0')
--------------------
tensor([[0.5723],
        [0.6746],
        [0.4676],
        [0.3640]], device='cuda:0')
tensor([[ True],
        [ True],
        [False],
        [False]], device='cuda:0')
--------------------
tensor([[0.5711],
        [0.6363],
        [0.3985],
        [0.6238]], device='cuda:0')
tensor([[ True],
        [ True],
        [False],
        [ True]], device='cuda:0')
--------------------
tensor([[0.6304],
        [0.7174],
        [0.5751],
        [0.6137]], device='cuda:0')
tensor([[True],
        [True],
        [True],
        [True]], device='cuda:0')
--------------------
tensor([[0.7460],
        [0.6951],
        [0.7555],
        [0.6498]], device='cuda:0')
tensor([[True],
        [True],
        [True],
        [True]], device='cuda:0')
--------------------
tensor([[0.8068],
        [

In [ ]:
# 모델 저장
torch.save(
    model,
    "models/250719_model.pt"
)

torch.save(
    model.state_dict(),
    "models/250719_model_state_dict"
)

In [ ]:
# 모델 불러오기

# 라이브러리
import torch
from torch import nn
import torch.serialization # 필요한 모듈 임포트

#커스텀 모델 정의
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(3, 1), # 입력3개 출력 1개인 '선형 변환' 레이어(가중치 학습)
            nn.Sigmoid() #활성화 함수 #0,1 사이 값으로 출력하는 '시그모이드 활성화' 함수
        )

    def forward(self, x): #모델의 순전파 정의 메서드
        x = self.layer(x) #정의된 레이러를 입력x에 순서대로 적용
        return x #최종 반환값

# GPU 연산 적용 및 모델 설정
device = "cuda" if torch.cuda.is_available() else "cpu" # GPU(CUDA) 사용 가능 여부 확인 후 디바이스 설정

# CustomModel과 torch.nn.Sequential, torch.nn.Linear, torch.nn.Sigmoid 클래스를 안전한 전역으로 등록
torch.serialization.add_safe_globals([CustomModel, torch.nn.Sequential, torch.nn.Linear, torch.nn.Sigmoid])

model = torch.load("models/250719_model.pt", map_location=device)
print(model)

CustomModel(
  (layer): Sequential(
    (0): Linear(in_features=3, out_features=1, bias=True)
    (1): Sigmoid()
  )
)


In [ ]:
# 모델 평가
with torch.no_grad(): # 평가 시 경사 계산 비활성화
    model.eval()     # 모델을 평가 모드로 전환 (Dropout, BatchNorm 등 영향)
    for x, y in validation_dataloader: # 검증 데이터로더에서 배치 가져오기
        x = x.to(device) # 입력 데이터를 디바이스로 이동
        y = y.to(device) # 레이블 데이터를 디바이스로 이동

        outputs = model(x) # 모델의 예측값 계산

        print(outputs) # 원본 예측 확률 출력
        # 0.5를 기준으로 이진 분류 결과 출력 (True/False)
        print(outputs >= torch.FloatTensor([0.5]).to(device))
        print("--------------------") # 배치 구분선

tensor([[4.7533e-23],
        [9.8041e-03],
        [5.3054e-28],
        [1.3380e-34]], device='cuda:0')
tensor([[False],
        [False],
        [False],
        [False]], device='cuda:0')
--------------------
tensor([[5.2723e-18],
        [2.1121e-14],
        [5.0499e-20],
        [7.2326e-18]], device='cuda:0')
tensor([[False],
        [False],
        [False],
        [False]], device='cuda:0')
--------------------
tensor([[7.4872e-21],
        [3.5017e-27],
        [1.5012e-30],
        [5.8616e-07]], device='cuda:0')
tensor([[False],
        [False],
        [False],
        [False]], device='cuda:0')
--------------------
tensor([[6.2297e-20],
        [3.1219e-26],
        [8.0370e-26],
        [9.7110e-05]], device='cuda:0')
tensor([[False],
        [False],
        [False],
        [False]], device='cuda:0')
--------------------
tensor([[6.8929e-04],
        [1.6930e-03],
        [5.2726e-27],
        [1.9183e-04]], device='cuda:0')
tensor([[False],
        [False],
        

In [ ]:
# 모델 구조확인

# 라이브러리
import torch
from torch import nn
import torch.serialization # 필요한 모듈 임포트

#커스텀 모델 정의
class CustomModel(nn.Module):
    pass

# GPU 연산 적용 및 모델 설정
device = "cuda" if torch.cuda.is_available() else "cpu" # GPU(CUDA) 사용 가능 여부 확인 후 디바이스 설정

# CustomModel과 torch.nn.Sequential, torch.nn.Linear, torch.nn.Sigmoid 클래스를 안전한 전역으로 등록
torch.serialization.add_safe_globals([CustomModel, torch.nn.Sequential, torch.nn.Linear, torch.nn.Sigmoid])

model = torch.load("models/250719_model.pt", map_location=device)
print(model)

CustomModel(
  (layer): Sequential(
    (0): Linear(in_features=3, out_features=1, bias=True)
    (1): Sigmoid()
  )
)


In [ ]:
# 모델의 상태 불러오기

# 라이브러리
import torch
from torch import nn


#커스텀 모델 정의
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(3, 1), # 입력3개 출력 1개인 '선형 변환' 레이어(가중치 학습)
            nn.Sigmoid() #활성화 함수 #0,1 사이 값으로 출력하는 '시그모이드 활성화' 함수
        )

    def forward(self, x): #모델의 순전파 정의 메서드
        x = self.layer(x) #정의된 레이러를 입력x에 순서대로 적용
        return x #최종 반환값

# GPU 연산 적용 및 모델 설정
device = "cuda" if torch.cuda.is_available() else "cpu" # GPU(CUDA) 사용 가능 여부 확인 후 디바이스 설정
model = CustomModel().to(device) # 모델 인스턴스 생성 및 디바이스로 이동

# 모델의 상태 불러오기
model.load_state_dict(torch.load("models/250719_model_state_dict"))

print(model)


CustomModel(
  (layer): Sequential(
    (0): Linear(in_features=3, out_features=1, bias=True)
    (1): Sigmoid()
  )
)


tensor([[6.8929e-04],
        [3.3257e-21],
        [2.0491e-19],
        [7.5278e-21]], device='cuda:0')
tensor([[False],
        [False],
        [False],
        [False]], device='cuda:0')
--------------------
tensor([[2.4695e-13],
        [8.0023e-17],
        [2.4420e-18],
        [7.0308e-24]], device='cuda:0')
tensor([[False],
        [False],
        [False],
        [False]], device='cuda:0')
--------------------
tensor([[7.3550e-09],
        [8.0370e-26],
        [5.7538e-28],
        [3.7211e-19]], device='cuda:0')
tensor([[False],
        [False],
        [False],
        [False]], device='cuda:0')
--------------------
tensor([[1.2066e-25],
        [1.3577e-21],
        [4.6476e-23],
        [1.9183e-04]], device='cuda:0')
tensor([[False],
        [False],
        [False],
        [False]], device='cuda:0')
--------------------
tensor([[1.3831e-08],
        [1.5027e-28],
        [6.2180e-28],
        [3.5017e-27]], device='cuda:0')
tensor([[False],
        [False],
        

In [ ]:
# 체크 포인트 불러오기

# 필요한 라이브러리 임포트
import torch # PyTorch 라이브러리
import pandas as pd # 데이터 처리 라이브러리
from torch import nn # 신경망 모듈
from torch import optim # 옵티마이저 모듈
from torch.utils.data import Dataset, DataLoader, random_split # 데이터 유틸리티

# 사용자 정의 데이터셋 클래스 정의
class CustomDataset(Dataset):
    def __init__(self, file_path):
        # CSV 파일 로드 및 특성(x)과 레이블(y) 분리
        df = pd.read_csv(file_path)
        self.x1 = df.iloc[:, 0].values
        self.x2 = df.iloc[:, 1].values
        self.x3 = df.iloc[:, 2].values
        self.y = df.iloc[:, 3].values
        self.length = len(df) # 데이터셋의 총 길이

    def __getitem__(self, index):
        # 특정 인덱스의 데이터와 레이블을 PyTorch 텐서로 반환
        x = torch.FloatTensor([self.x1[index], self.x2[index], self.x3[index]])
        y = torch.FloatTensor([int(self.y[index])])
        return x, y

    def __len__(self):
        # 데이터셋의 총 길이 반환
        return self.length

# 사용자 정의 모델 클래스 정의
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__() # 부모 클래스 초기화
        # 선형 레이어와 시그모이드 활성화 함수로 구성된 순차적 모델 정의
        self.layer = nn.Sequential(
            nn.Linear(3, 1), # 입력 3개, 출력 1개인 선형 레이어
            nn.Sigmoid()     # 0과 1 사이의 값으로 변환하는 시그모이드 활성화 함수
        )

    def forward(self, x):
        # 모델의 순전파 로직 정의
        x = self.layer(x)
        return x

# 데이터셋 로드, 분할 및 데이터로더 생성
dataset = CustomDataset("./datasets/binary.csv") # CustomDataset 객체 생성 (데이터 파일 경로 지정)
dataset_size = len(dataset)                       # 전체 데이터셋 크기
train_size = int(dataset_size * 0.8)              # 훈련 세트 크기 (80%)
validation_size = int(dataset_size * 0.1)         # 검증 세트 크기 (10%)
test_size = dataset_size - train_size - validation_size # 테스트 세트 크기 (나머지)

# 데이터셋을 훈련, 검증, 테스트 세트로 무작위 분할 (재현성을 위해 시드 고정)
train_dataset, validation_dataset, test_dataset = random_split(
    dataset, [train_size, validation_size, test_size], torch.manual_seed(4)
)

# DataLoader를 사용하여 각 데이터셋을 미니 배치 단위로 준비
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=4, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=True, drop_last=True)

# GPU 연산 적용 및 모델 설정
device = "cuda" if torch.cuda.is_available() else "cpu" # GPU(CUDA) 사용 가능 여부 확인 후 디바이스 설정
model = CustomModel().to(device)                       # CustomModel 인스턴스 생성 및 디바이스로 이동
criterion = nn.BCELoss().to(device)                    # 이진 교차 엔트로피 손실 함수 정의 및 디바이스로 이동
optimizer = optim.SGD(model.parameters(), lr=0.0001)   # SGD 옵티마이저 정의 (모델 파라미터와 학습률 지정)

#체크포인트 불러오기
checkpoint = torch.load("./models/checkpoint-10.pt")
model.load_state_dict(checkpoint["model_state_dict"])
optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
checkpoint_epoch = checkpoint["epoch"]
checkpoint_description = checkpoint["description"]
print(checkpoint_description)

# 모델 학습 진행 (Training Loop)
for epoch in range(10000): # 총 10,000 에포크 동안 학습 반복
    cost = 0.0 # 현재 에포크의 누적 손실 초기화

    for x, y in train_dataloader: # 훈련 데이터로더에서 배치 단위로 데이터 가져오기
        x = x.to(device) # 입력 데이터를 디바이스로 이동
        y = y.to(device) # 레이블 데이터를 디바이스로 이동

        output = model(x) # 순전파: 모델을 통해 예측값 계산
        loss = criterion(output, y) # 손실 계산: 예측값과 실제값 비교

        optimizer.zero_grad() # 이전 경사값 초기화
        loss.backward()       # 역전파: 손실에 대한 경사 계산
        optimizer.step()      # 파라미터 업데이트: 계산된 경사를 이용해 모델 가중치 조정

        cost += loss # 현재 배치 손실을 누적

    cost = cost / len(train_dataloader) # 에포크의 평균 손실 계산

    if (epoch + 1) % 1000 == 0: # 1000 에포크마다 진행 상황 출력
        print(f"Epoch : {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost:.3f}")

CustomModel 체크포인트-10
Epoch : 1000, Model : [Parameter containing:
tensor([[ 0.0028, -0.0006,  0.0036]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.0949], device='cuda:0', requires_grad=True)], Cost : 0.680
Epoch : 2000, Model : [Parameter containing:
tensor([[0.0034, 0.0005, 0.0041]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([-0.0346], device='cuda:0', requires_grad=True)], Cost : 0.666
Epoch : 3000, Model : [Parameter containing:
tensor([[0.0045, 0.0017, 0.0051]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([-0.1609], device='cuda:0', requires_grad=True)], Cost : 0.652
Epoch : 4000, Model : [Parameter containing:
tensor([[0.0052, 0.0017, 0.0053]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([-0.2843], device='cuda:0', requires_grad=True)], Cost : 0.636
Epoch : 5000, Model : [Parameter containing:
tensor([[0.0059, 0.0036, 0.0064]], device='cuda:0', requires_grad=True), Parameter containin

In [ ]:
# 코드 전체 묶음

In [5]:
# 필요한 라이브러리 임포트
import torch # PyTorch 라이브러리
import pandas as pd # 데이터 처리 라이브러리
from torch import nn # 신경망 모듈
from torch import optim # 옵티마이저 모듈
from torch.utils.data import Dataset, DataLoader, random_split # 데이터 유틸리티

# 사용자 정의 데이터셋 클래스 정의
class CustomDataset(Dataset):
    def __init__(self, file_path):
        # CSV 파일 로드 및 특성(x)과 레이블(y) 분리
        df = pd.read_csv(file_path)
        self.x1 = df.iloc[:, 0].values
        self.x2 = df.iloc[:, 1].values
        self.x3 = df.iloc[:, 2].values
        self.y = df.iloc[:, 3].values
        self.length = len(df) # 데이터셋의 총 길이

    def __getitem__(self, index):
        # 특정 인덱스의 데이터와 레이블을 PyTorch 텐서로 반환
        x = torch.FloatTensor([self.x1[index], self.x2[index], self.x3[index]])
        y = torch.FloatTensor([int(self.y[index])])
        return x, y

    def __len__(self):
        # 데이터셋의 총 길이 반환
        return self.length

# 사용자 정의 모델 클래스 정의
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__() # 부모 클래스 초기화
        # 선형 레이어와 시그모이드 활성화 함수로 구성된 순차적 모델 정의
        self.layer = nn.Sequential(
            nn.Linear(3, 1), # 입력 3개, 출력 1개인 선형 레이어
            nn.Sigmoid()     # 0과 1 사이의 값으로 변환하는 시그모이드 활성화 함수
        )

    def forward(self, x):
        # 모델의 순전파 로직 정의
        x = self.layer(x)
        return x

# 데이터셋 로드, 분할 및 데이터로더 생성
dataset = CustomDataset("./datasets/binary.csv") # CustomDataset 객체 생성 (데이터 파일 경로 지정)
dataset_size = len(dataset)                       # 전체 데이터셋 크기
train_size = int(dataset_size * 0.8)              # 훈련 세트 크기 (80%)
validation_size = int(dataset_size * 0.1)         # 검증 세트 크기 (10%)
test_size = dataset_size - train_size - validation_size # 테스트 세트 크기 (나머지)

# 데이터셋을 훈련, 검증, 테스트 세트로 무작위 분할 (재현성을 위해 시드 고정)
train_dataset, validation_dataset, test_dataset = random_split(
    dataset, [train_size, validation_size, test_size], torch.manual_seed(4)
)

# DataLoader를 사용하여 각 데이터셋을 미니 배치 단위로 준비
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=4, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=True, drop_last=True)

# GPU 연산 적용 및 모델 설정
device = "cuda" if torch.cuda.is_available() else "cpu" # GPU(CUDA) 사용 가능 여부 확인 후 디바이스 설정
model = CustomModel().to(device)                       # CustomModel 인스턴스 생성 및 디바이스로 이동
criterion = nn.BCELoss().to(device)                    # 이진 교차 엔트로피 손실 함수 정의 및 디바이스로 이동
optimizer = optim.SGD(model.parameters(), lr=0.0001)   # SGD 옵티마이저 정의 (모델 파라미터와 학습률 지정)

# 모델 학습 진행 (Training Loop)
for epoch in range(10000): # 총 10,000 에포크 동안 학습 반복
    cost = 0.0 # 현재 에포크의 누적 손실 초기화

    for x, y in train_dataloader: # 훈련 데이터로더에서 배치 단위로 데이터 가져오기
        x = x.to(device) # 입력 데이터를 디바이스로 이동
        y = y.to(device) # 레이블 데이터를 디바이스로 이동

        output = model(x) # 순전파: 모델을 통해 예측값 계산
        loss = criterion(output, y) # 손실 계산: 예측값과 실제값 비교

        optimizer.zero_grad() # 이전 경사값 초기화
        loss.backward()       # 역전파: 손실에 대한 경사 계산
        optimizer.step()      # 파라미터 업데이트: 계산된 경사를 이용해 모델 가중치 조정

        cost += loss # 현재 배치 손실을 누적

    cost = cost / len(train_dataloader) # 에포크의 평균 손실 계산

    if (epoch + 1) % 1000 == 0: # 1000 에포크마다 진행 상황 출력
        print(f"Epoch : {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost:.3f}")

# 모델 평가
with torch.no_grad(): # 평가 시 경사 계산 비활성화
    model.eval()     # 모델을 평가 모드로 전환 (Dropout, BatchNorm 등 영향)
    for x, y in validation_dataloader: # 검증 데이터로더에서 배치 가져오기
        x = x.to(device) # 입력 데이터를 디바이스로 이동
        y = y.to(device) # 레이블 데이터를 디바이스로 이동

        outputs = model(x) # 모델의 예측값 계산

        print(outputs) # 원본 예측 확률 출력
        # 0.5를 기준으로 이진 분류 결과 출력 (True/False)
        print(outputs >= torch.FloatTensor([0.5]).to(device))
        print("--------------------") # 배치 구분선

# 모델 저장
# 전체 모델 저장 (환경 종속성 있음, 개발/테스트용)
torch.save(
    model,
    ".models/250719_model.pt"
)

# 모델의 state_dict (파라미터만) 저장 (권장되는 방법, 유연성 높음)
torch.save(
    model.state_dict(),
    ".models/250719_model_state_dict"
)

Epoch : 1000, Model : [Parameter containing:
tensor([[ 0.0028, -0.0006,  0.0036]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.0949], device='cuda:0', requires_grad=True)], Cost : 0.680
Epoch : 2000, Model : [Parameter containing:
tensor([[0.0034, 0.0005, 0.0041]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([-0.0346], device='cuda:0', requires_grad=True)], Cost : 0.666
Epoch : 3000, Model : [Parameter containing:
tensor([[0.0045, 0.0017, 0.0051]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([-0.1609], device='cuda:0', requires_grad=True)], Cost : 0.652
Epoch : 4000, Model : [Parameter containing:
tensor([[0.0052, 0.0017, 0.0053]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([-0.2843], device='cuda:0', requires_grad=True)], Cost : 0.636
Epoch : 5000, Model : [Parameter containing:
tensor([[0.0059, 0.0036, 0.0064]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([-0.4046], 

RuntimeError: Parent directory .models does not exist.

In [ ]:
# 모델 저장
# 전체 모델 저장 (환경 종속성 있음, 개발/테스트용)
torch.save(
    model,
    "models/250719_model.pt"
)
'''
# 모델의 state_dict (파라미터만) 저장 (권장되는 방법, 유연성 높음)
torch.save(
    model.state_dict(),
    "models/250719_model_state_dict"
)
'''